https://leetcode.com/problems/wildcard-matching/

Given an input string (s) and a pattern (p), implement wildcard pattern matching with support for '?' and '*' where:

    '?' Matches any single character.
    '*' Matches any sequence of characters (including the empty sequence).

The matching should cover the entire input string (not partial).

In [1]:
import os
os.chdir('../useful_modules')
from modules import *

In [2]:
# Fail
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        while '**' in p:
            p = p.replace('**', '*')
        # p = p.replace('*?', '*')
        return self.helper(s, p)
    
    def helper(self, s, p):
        if len(p) == 0 and len(s) > 0:
            return False
        elif (len(p) + len(s)) == 0:
            return True
        
        if p[0] == '*':
            if len(s) > 0:
                if len(p) > 1 and p[1] != '?':
                    i = s.find(p[1])
                    if i == -1:
                        return False
                    elif i != 0:
                        return self.helper(s[i:], p)
                    
                
                return self.helper(s[1:], p[1:]) or self.helper(s, p[1:]) or self.helper(s[1:], p)
            else:
                return self.helper(s, p[1:])
        elif p[0] == '?':
            if len(s) > 0:
                return self.helper(s[1:], p[1:])
            else:
                return False
        else:
            if len(s) > 0:
                if s[0] == p[0]:
                    return self.helper(s[1:], p[1:])
                else:
                    return False
            else:
                return False

In [3]:
# Fail

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        queue = []
        for term in p:
            if term == '*':
                if queue:
                    if queue[-1] != '*':
                        queue.append('*')
                else:
                    queue.append('*')
            elif '?' == term:
                queue.append('?')
            else:
                if queue and (not queue[-1] in ['*', '?']):
                    queue[-1] += term
                else:
                    queue.append(term)
        return self.setMatch(s, queue)
    
    def setMatch(self, s, queue):
        if len(queue) == 0 and len(s) > 0:
            return False
        elif (len(queue) + len(s)) == 0:
            return True
        elif len(''.join(queue).replace('*','')) > len(s):
            return False
        
        if queue[0] == '*' and len(queue) == 1:
            return True
        elif queue[0] == '*' and len(queue) > 1:
            if queue[1] == '?':
                return self.setMatch(s[1:], queue[2:]) or self.setMatch(s, queue[1:]) or self.setMatch(s[1:], queue)
            else:
                res = s.find(queue[1])
                if res == -1:
                    return False
                elif res > 0:
                    return self.setMatch(s[res:], queue)
                else:
                    l = len(queue[1])
                    return self.setMatch(s[l:], queue[2:]) or self.setMatch(s, queue[1:]) or self.setMatch(s[1:], queue)
        elif queue[0] == '?':
            return self.setMatch(s[1:], queue[1:])
        else:
            if s.startswith(queue[0]):
                return self.setMatch(s[len(queue[0]):], queue[1:])
            else:
                return False


In [4]:
# Official
# Runtime 78.95%
# Memory 63.76%

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        s_len = len(s)
        p_len = len(p)
        
        # base cases
        if p == s or set(p) == {'*'}:
            return True
        if p == '' or s == '':
            return False
        
        # init all matrix except [0][0] element as False
        d = [[False] * (s_len + 1) for _ in range(p_len + 1)]
        d[0][0] = True
        
        # DP compute 
        for p_idx in range(1, p_len + 1):
            # the current character in the pattern is '*'
            if p[p_idx - 1] == '*':
                s_idx = 1
                                        
                # d[p_idx - 1][s_idx - 1] is a string-pattern match 
                # on the previous step, i.e. one character before.
                # Find the first idx in string with the previous math.
                while not d[p_idx - 1][s_idx - 1] and s_idx < s_len + 1:
                    s_idx += 1
    
                # If (string) matches (pattern), 
                # when (string) matches (pattern)* as well
                d[p_idx][s_idx - 1] = d[p_idx - 1][s_idx - 1]
    
                # If (string) matches (pattern), 
                # when (string)(whatever_characters) matches (pattern)* as well
                while s_idx < s_len + 1:
                    d[p_idx][s_idx] = True
                    s_idx += 1
                                   
            # the current character in the pattern is '?'
            elif p[p_idx - 1] == '?':
                for s_idx in range(1, s_len + 1): 
                    d[p_idx][s_idx] = d[p_idx - 1][s_idx - 1] 
                                   
            # the current character in the pattern is not '*' or '?'
            else:
                for s_idx in range(1, s_len + 1): 
                    # Match is possible if there is a previous match
                    # and current characters are the same
                    d[p_idx][s_idx] = d[p_idx - 1][s_idx - 1] and p[p_idx - 1] == s[s_idx - 1]  
                                                               
        return d[p_len][s_len]

In [5]:
# Runtime: 22.32%
# Memory: 33.65%
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        d = []
        star_queue = [[], []]
        for j in range(len(p)+1):
            star_queue.pop(0)
            star_queue.append([])
            for i in range(len(s)+1):
                # print(star_queue)
                if i == 0 and j == 0:
                    d.append([True])
                elif i > 0 and j == 0:
                    d.append([False])
                elif i == 0 and j > 0:
                    d[i].append(d[i][j-1] and p[j-1] == '*')
                else:
                    normal = (p[j-1] == s[i-1]) and d[i-1][j-1]
                    question = (p[j-1] == '?') and d[i-1][j-1]
                    star = (p[j-1] == '*') and star_queue[0][i]
#                     if p[j-1] == '*': 
#                         for i_ in range(i+1):
#                             star = star or d[i_][j-1]
#                             if star: break
                    d[i].append(normal or question or star)  
                if i == 0:
                    # print(star_queue)
                    # print(i,j,d[i][j])
                    star_queue[1].append(d[i][j])
                else:
                    # print(star_queue)
                    star_queue[1].append(d[i][j] or star_queue[1][-1])
#         for i in range(len(d)):
#             print(d[i])
        return d[i][j]

In [6]:
inputs = [
    ['aa', 'a'], # False
    ['aa', '*'], # True
    ['cb', '?a'], # FAlse
    ["adceb", "*a*b"], # True
    ["aaaa", '***a'], # True
    ["aaabababaaabaababbbaaaabbbbbbabbbbabbbabbaabbababab", "*ab***ba**b*b*aaab*b"], # True
    ["babaaababaabababbbbbbaabaabbabababbaababbaaabbbaaab", "***bba**a*bbba**aab**b"], # False
    ["mississippi", "m??*ss*?i*pi"], # False
    ["babbbaabbaaaaabbababaaaabbbbbbbbbbabbaaaabbababbabaa", "**a****a**b***ab***a*bab"], # False
    ["baaaa", '*aaa'], # True
    ["abcabczzzde", "*abc???de*"], # True
]

In [7]:
test(inputs, Solution().isMatch)

aa a : False
aa * : True
cb ?a : False
adceb *a*b : True
aaaa ***a : True
aaabababaaabaababbbaaaabbbbbbabbbbabbbabbaabbababab *ab***ba**b*b*aaab*b : True
babaaababaabababbbbbbaabaabbabababbaababbaaabbbaaab ***bba**a*bbba**aab**b : False
mississippi m??*ss*?i*pi : False
babbbaabbaaaaabbababaaaabbbbbbbbbbabbaaaabbababbabaa **a****a**b***ab***a*bab : False
baaaa *aaa : True
abcabczzzde *abc???de* : True
Time elapsed: 0.00214 seconds
